In [1]:
import pandas as pd
import json
import ast

# preparing housing data from csv so we can load it an run it with a clustering algorithm
# code below reads the final attribute in the csv file which stores a json dump of data
filename = "housing_data.csv"

col_list = ["Lat", "Lon", "Price"]


# splitting data frame in half, ~50% for training ~50% for testing
df = pd.read_csv(filename, usecols=col_list)
df_train = df.head(6868)
df_test = df.tail(1717)


len(df_train.index)
len(df_test.index)


1717

In [2]:
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=15, random_state=1).fit(df_train)


In [3]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(20,10))
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(df_train["Lat"],df_train["Lon"],df_train["Price"], c=kmeans.labels_, cmap='viridis',edgecolor='k', s=40, alpha = 0.5)

# ax.dist = 10

# # ax.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,2], s = 300, c = 'r', marker='*', label = 'Centroid')

# plt.autoscale(enable=True, axis='x', tight=True)


In [4]:
# adding the mean of price of the housing cluster in addition to the location of the cluster centre

# centroids=kmeans.cluster_centers_

# print(centroids.shape)

# #iterating through the list of centroids
# index = 0
# for cen in centroids:

#     centroid = df_train[kmeans.labels_==index]


#     centroids[2] = centroid["Price"].mean()
#     print("\nLat ",cen[0],"\nLon", cen[1],"\nPrice ", cen[2])
#     index = index + 1


In [5]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=85, random_state=1).fit(df_train)


# adding the mean of price of the housing cluster in addition to the location of the cluster centre

centroids = kmeans.cluster_centers_

# print(centroids.shape)

# iterating through the list of centroids
index = 0
for cen in centroids:

    centroid = df_train[kmeans.labels_ == index]

    centroids[2] = centroid["Price"].mean()
    #     print("\nLat ",cen[0],"\nLon", cen[1],"\nPrice ", cen[2])
    index = index + 1


import geopy.distance
import numpy as np

from haversine import haversine, Unit

# coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

ar = np.zeros((4293, 1))

# print( geopy.distance.vincenty(coords_1, coords_2).km)

# print(geopy.distance.distance(coords_1, coords_2).km)


# go through each row and assign it to the closest centroid

i = 0
for row in df_test.iterrows():

    # we have a test point
    coordinates_of_test_point = (row[1]["Lat"], row[1]["Lon"])

    # set a huge minimum distance
    mini = 1000000000

    # index so we can select the correct centroid
    index = 0
    # iterate through the centroids
    for centroid in centroids:

        # grab the coords of the centroid
        coordinates_of_centroid = (centroid[0], centroid[1])

        # calculate the distance between the test point and the centroid
        #         dist = geopy.distance.distance(coordinates_of_test_point, coordinates_of_centroid).km
        dist = haversine(coordinates_of_test_point, coordinates_of_centroid)
        closest_centroid_index = 0
        # if the dist is smaller then min
        if dist < mini:
            # set mini to that distance
            mini = dist
            # set that centroid to be the one the test point is associated with and store that index
            closest_centroid_index = index

        #         centroids[closest_centroid_index][2]
        ar[i] = abs(centroids[closest_centroid_index][2] - row[1]["Price"])

        index = index + 1
    i = i + 1


print(np.mean(ar))


232491.43666904472


In [6]:
from sklearn.cluster import KMeans


def k_mean(num_c):

    kmeans = KMeans(n_clusters=num_c, random_state=1).fit(df_train)

    # adding the mean of price of the housing cluster in addition to the location of the cluster centre

    centroids = kmeans.cluster_centers_

    # print(centroids.shape)

    # iterating through the list of centroids
    index = 0
    for cen in centroids:

        centroid = df_train[kmeans.labels_ == index]

        centroids[2] = centroid["Price"].mean()
        #     print("\nLat ",cen[0],"\nLon", cen[1],"\nPrice ", cen[2])
        index = index + 1

    import geopy.distance
    import numpy as np

    from haversine import haversine, Unit

    # coords_1 = (52.2296756, 21.0122287)
    coords_2 = (52.406374, 16.9251681)

    ar = np.zeros((4293, 1))

    # print( geopy.distance.vincenty(coords_1, coords_2).km)

    # print(geopy.distance.distance(coords_1, coords_2).km)

    # go through each row and assign it to the closest centroid

    i = 0
    for row in df_test.iterrows():

        # we have a test point
        coordinates_of_test_point = (row[1]["Lat"], row[1]["Lon"])

        # set a huge minimum distance
        mini = 1000000000

        # index so we can select the correct centroid
        index = 0
        # iterate through the centroids
        for centroid in centroids:

            # grab the coords of the centroid
            coordinates_of_centroid = (centroid[0], centroid[1])

            # calculate the distance between the test point and the centroid
            #         dist = geopy.distance.distance(coordinates_of_test_point, coordinates_of_centroid).km
            dist = haversine(coordinates_of_test_point, coordinates_of_centroid)
            closest_centroid_index = 0
            # if the dist is smaller then min
            if dist < mini:
                # set mini to that distance
                mini = dist
                # set that centroid to be the one the test point is associated with and store that index
                closest_centroid_index = index

            #         centroids[closest_centroid_index][2]
            ar[i] = abs(centroids[closest_centroid_index][2] - row[1]["Price"])

            index = index + 1
        i = i + 1
    #     print(np.mean(ar))

    return num_c, np.mean(ar)


k_mean(3)
# print(np.mean(ar))


(3, 184766.01431236695)

In [ ]:
clusters_pricediff = np.zeros((1000, 2))

for i in range(3, 1000):
    num_c, avg_price_diff = k_mean(i)

    clusters_pricediff[i][0] = num_c
    clusters_pricediff[i][1] = avg_price_diff


In [ ]:
ar = []
i = 0

for row in clusters_pricediff:

    if i == 0 or i == 1 or i == 2:
        print("")
    else:
        ar.append(row[1])
    i = i + 1


min(ar)
